In [1]:
import os
import datetime

import numpy as np

import matplotlib as mpl
from matplotlib.collections import PolyCollection
from matplotlib.patches import Polygon
from matplotlib import pyplot as plt

import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

%matplotlib inline

ModuleNotFoundError: No module named 'cartopy'

In [ ]:
plt.rcParams['font.size']      = 18
plt.rcParams['font.weight']    = 'bold'
plt.rcParams['axes.grid']      = True
plt.rcParams['axes.xmargin']   = 0
plt.rcParams['grid.linestyle'] = ':'

In [ ]:
fig     = plt.figure(figsize=figsize)
            ax      = fig.add_subplot(111,projection=ccrs.PlateCarree())

    #        ax.coastlines(zorder=10,color='k')
    #        ax.add_feature(cartopy.feature.LAND)
    #        ax.add_feature(cartopy.feature.OCEAN)
    #        ax.add_feature(cartopy.feature.LAKES, alpha=0.5)
    #        ax.add_feature(cartopy.feature.RIVERS)
            ax.add_feature(cartopy.feature.COASTLINE)
            ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
            ax.set_title('')
            ax.gridlines(draw_labels=True)

            LONS, LATS  = np.meshgrid(lons,lats)
            data        = this_edens[:,:,alt_inx]
            pcoll       = ax.pcolormesh(LONS,LATS,data,cmap=cmap,norm=norm)

            if plot_profile_paths == 'all':
                for prof_key,profile in self.profiles.items():
                    glons       = profile['glons']
                    tf          = glons > 180
                    glons[tf]   = glons[tf]-360.
                    glats       = profile['glats']
                    ax.plot(glons,glats,marker='o',color='k',lw=4,zorder=100)

                    tx_call     = profile.attrs['tx_call']
                    tx_lat      = profile.attrs['tx_lat']
                    tx_lon      = profile.attrs['tx_lon']
                    if tx_lon > 180:
                        tx_lon = tx_lon - 360.

                    rx_call     = profile.attrs['rx_call']
                    rx_lat      = profile.attrs['rx_lat']
                    rx_lon      = profile.attrs['rx_lon']
                    if rx_lon > 180:
                        rx_lon = rx_lon - 360.

                    ax.scatter(tx_lon,tx_lat,marker='*',s=450,zorder=110,label=tx_call,ec='k',fc='red')
                    ax.scatter(rx_lon,rx_lat,marker='*',s=450,zorder=110,label=rx_call,ec='k',fc='orange')
                    fontdict = {'size':'x-large','weight':'bold'}
                    offset = 1.1
                    ax.text(tx_lon,tx_lat+offset,tx_call,fontdict=fontdict,ha='center')
                    ax.text(rx_lon,rx_lat+offset,rx_call,fontdict=fontdict,ha='center')

            # Plot Wave Source Locations
            if hasattr(self,'wave_list'):
                for wave in self.wave_list:
                    wlat    = wave['src_lat']
                    wlon    = wave['src_lon']
                    ax.scatter(wlon,wlat,marker='^',s=100,zorder=110,ec='k',fc='red')

            ax.set_xlim(xlim)
            ax.set_ylim(ylim)